In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")  # Go up one level to the UROP directory

In [ ]:
import yaml

with open("config.yaml") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

batch_size = config["batch_size"]
lr = config["lr"]
num_epochs = config["num_epochs"]
save_every = config["save_every"]
save_dir = config["save_dir"]
num_workers = config["num_workers"]

In [ ]:
import torch
from torch.utils.data import DataLoader
from utils import LazyWindowedDataset

test_source_dataset = LazyWindowedDataset(
    root_dir="datasets/BrushlessMotor",
    split="test",
    anomaly_type=['normal', 'anomaly'],
    domain_type=['source', 'target'],
    window_size_ms=100,
    stride_ms=50,
)
test_loader = DataLoader(test_source_dataset, batch_size=batch_size, shuffle=True, num_workers=1, drop_last=True)

In [6]:
from models import BaselineAutoencoder
from utils import evaluate, load_model_checkpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, _, _, _, _, _ = load_model_checkpoint("checkpoints/BrushlessMotorBaseline.pt", BaselineAutoencoder, optimizer_class=None)
criterion = torch.nn.MSELoss()

![alt text](figures/brushlessMotorHighlightedTable.png)

In [10]:
loss, auc = evaluate(model, test_loader, criterion)
print(f"Final loss: {loss:.4f}, AUC: {auc:.4f}")

Final loss: 0.8568, AUC: 0.5884


In [7]:
acc_loss, acc_auc = evaluate(model, test_loader, criterion, sensors_to_test=['acc'])
print(f"Final loss: {acc_loss:.4f}, AUC: {acc_auc:.4f}")

Final loss: 0.4075, AUC: 0.6799


In [8]:
gyro_loss, gyro_auc = evaluate(model, test_loader, criterion, sensors_to_test=['gyro'])
print(f"Final loss: {gyro_loss:.4f}, AUC: {gyro_auc:.4f}")

Final loss: 0.2563, AUC: 0.5709


In [9]:
mic_loss, mic_auc = evaluate(model, test_loader, criterion, sensors_to_test=['mic'])
print(f"Final loss: {mic_loss:.4f}, AUC: {mic_auc:.4f}")

Final loss: 0.3634, AUC: 0.4804
